In [ ]:
# !pip install py2neo

In [1]:
import os, requests, json, gzip, json
from py2neo import Graph

DIR = "./"
FILE = "conceptnet-assertions-5.7.0.csv.gz"

HOST = "neo4j"
neo4jauth = tuple(os.environ["NEO4J_AUTH"].split("/"))
graph = Graph(host=HOST, auth=(neo4jauth)) 

In [1]:
# Build query.
query = """
WITH $json AS document
UNWIND document.edges AS edges
WITH 
SPLIT(edges.start,"/")[3] AS startConcept,
SPLIT(edges.start,"/")[2] AS startLanguage,
CASE WHEN SPLIT(edges.start,"/")[4] <> "" THEN SPLIT(edges.start,"/")[4] ELSE "" END AS startPartOfSpeech,
CASE WHEN SPLIT(edges.start,"/")[5] <> "" THEN SPLIT(edges.start,"/")[5] ELSE "" END AS startSense,
SPLIT(edges.rel,"/")[2] AS relType,
CASE WHEN edges.surfaceText <> "" THEN edges.surfaceText ELSE "" END AS surfaceText, edges.weight AS weight,
SPLIT(edges.end,"/")[3] AS endConcept,
SPLIT(edges.end,"/")[2] AS endLanguage,
CASE WHEN SPLIT(edges.end,"/")[4] <> "" THEN SPLIT(edges.end,"/")[4] ELSE "" END AS endPartOfSpeech,
CASE WHEN SPLIT(edges.end,"/")[5] <> "" THEN SPLIT(edges.end,"/")[5] ELSE "" END AS endSense
MERGE (start:Term {concept:startConcept, language:startLanguage, partOfSpeech:startPartOfSpeech, sense:startSense})
MERGE (end:Term  {concept:endConcept, language:endLanguage, partOfSpeech:endPartOfSpeech, sense:endSense})
MERGE (start)-[r:ASSERTION {type:relType, weight:weight, surfaceText:surfaceText}]-(end)
"""

DELETE_Q = "MATCH (n) DETACH DELETE n"
graph.run(DELETE_Q)

NameError: name 'graph' is not defined

In [3]:
def insert_cn2neo4j():
    data = []
    i = 0
    with gzip.open(DIR + FILE, mode='rt') as fp:
        line = fp.readline()
        while line:
            uri, rel, c1, c2, info = line.split("\t") 
            langs = { c1.split("/")[2], c2.split("/")[2] }
            if "ja" in langs:
                info = json.loads(info)
                info["rel"] = rel
                info["start"] = c1
                info["end"] = c2
                data.append(info)
                i += 1
                if i % 1000 == 0:
                    input_json = {"edges": data}
                    graph.run(query, json=input_json)
                    data = []
                    if i % 10000 == 0:
                        print(f"{i/1631492*100}", end=", ")
            line = fp.readline()
    print(i)

In [4]:
%%time
insert_cn2neo4j()

KeyboardInterrupt: 

In [6]:
%%time
insert_cn2neo4j()

1631492
CPU times: user 2min 27s, sys: 753 ms, total: 2min 28s
Wall time: 2min 28s
